# Train YOLOv4 using tflite
https://github.com/hunglc007/tensorflow-yolov4-tflite


# PyTorch YOLOv4
    https://github.com/WongKinYiu/PyTorch_YOLOv4
    https://github.com/maudzung/Complex-YOLOv4-Pytorch
    https://github.com/Tianxiaomo/pytorch-YOLOv4#22-image-input-size-for-inference
    https://github.com/WongKinYiu/PyTorch_YOLOv4
    https://github.com/VCasecnikovs/Yet-Another-YOLOv4-Pytorch

In [105]:
import os
import subprocess
import pandas as pd
import cv2

In [106]:
os.chdir('C:\DS_ML\Video_Analytics_YOLOv4\Model_Retraining')

In [110]:
import yoloDataPrep as ydp

In [108]:
# Read the meta data from database in dataframe
trainmd, testmd = ydp.readMetaData()
trainmd


Training Table Present and contains 216 videos
Testing Table Present and contains 93 videos


,video_name,video_path,object_path,event_path
0,VIRAT_S_050300_04_001057_001122.mp4,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...
1,VIRAT_S_050202_07_001126_001301.mp4,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...
2,VIRAT_S_010003_10_000901_000934.mp4,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...
3,VIRAT_S_000201_06_001354_001397.mp4,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...
4,VIRAT_S_000201_00_000018_000380.mp4,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...
...,...,...,...,...
211,VIRAT_S_010205_04_000545_000576.mp4,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...
212,VIRAT_S_010003_09_000779_000861.mp4,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...
213,VIRAT_S_010109_05_000602_000678.mp4,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,None
214,VIRAT_S_050000_11_001530_001576.mp4,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...


In [109]:
# Read annotation data from database in dataframe
trainad, testad = ydp.readAnnotData()
trainad

Training annotaions table is present and contains 1568499 objects
Testing annotation table is present and contains 670980 objects


,obj_id,frame_num,bb_lt_x,bb_lt_y,bb_width,bb_height,obj_type,video_name
0,3,0,749,448,29,56,1,VIRAT_S_050300_04_001057_001122.mp4
1,3,1,746,448,32,56,1,VIRAT_S_050300_04_001057_001122.mp4
2,3,2,746,448,32,56,1,VIRAT_S_050300_04_001057_001122.mp4
3,3,3,746,448,32,56,1,VIRAT_S_050300_04_001057_001122.mp4
4,3,4,746,448,32,56,1,VIRAT_S_050300_04_001057_001122.mp4
...,...,...,...,...,...,...,...,...
1568494,98,1823,0,193,12,32,1,VIRAT_S_010108_01_000570_000718.mp4
1568495,98,1824,0,193,12,32,1,VIRAT_S_010108_01_000570_000718.mp4
1568496,98,1825,0,193,12,32,1,VIRAT_S_010108_01_000570_000718.mp4
1568497,98,1826,0,193,12,32,1,VIRAT_S_010108_01_000570_000718.mp4


In [29]:
# As the data volume is too large, we will split the data in 10 batches
trainmb, testmb = ydp.miniBatchData(traindf = trainmd, testdf = testmd, batches = 10)
print('No. of videos in first training batch: {}'.format(len(trainmb[trainmb.batchno == 1])))
print('No. of videos in first testing batch: {}'.format(len(testmb[testmb.batchno == 1])))

No. of videos in first training batch: 22
No. of videos in first testing batch: 10


In [30]:
# Training videos
trainmb[trainmb.batchno == 1]


,video_name,video_path,object_path,event_path,batchno
0,VIRAT_S_050300_04_001057_001122.mp4,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,1
1,VIRAT_S_050202_07_001126_001301.mp4,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,1
2,VIRAT_S_010003_10_000901_000934.mp4,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,1
3,VIRAT_S_000201_06_001354_001397.mp4,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,1
4,VIRAT_S_000201_00_000018_000380.mp4,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,1
5,VIRAT_S_050000_16_001947_001989.mp4,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,1
6,VIRAT_S_040004_00_000011_000093.mp4,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,None,1
7,VIRAT_S_010106_04_000800_000918.mp4,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,1
8,VIRAT_S_010003_06_000526_000560.mp4,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,1
9,VIRAT_S_040005_06_000886_001016.mp4,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,1


In [12]:
# No. of training images
print('Total number of training images that contains bounding box: {}'.format(len(trainad[trainad.video_name.isin(trainmb[trainmb.batchno == 1]['video_name'])].groupby(['video_name', 'frame_num']).count())))



Total number of training images that contains bounding box: 38012


###       Observations:-
    * Total number of images with a person in it is 38,012 
    * Actual number of training images is same as total frame count of all the selected videos and that is 49,871  
    * This also means that we have 49,871 - 38,012 = 11,859 images as negative sales, i.e w/o bounding box and blank annotation file. This is in-line with the recommendation to improve object detection 

In [13]:
# Missing video from trainad dataframe
trainad[trainad.video_name == 'VIRAT_S_040004_00_000011_000093.mp4']


,obj_id,frame_num,bb_lt_x,bb_lt_y,bb_width,bb_height,obj_type,video_name


### Observation:-
Video VIRAT_S_040004_00_000011_000093.mp4 was present in the Metadata table however was not present in annotations data table. This was becasue annotation data table contains filtered annotation for object of interest, in our case 'Person', which is what we want our model to train on. This particular video doesn't contain any person. Just has car. It might be a good candidate for traffic survilience application but not good for human detection.

In [16]:
# Generate YOLO Training data
dataFolder = 'C:/DS_ML/Video_Analytics_YOLOv4/YOLOv4/darknet/build/darknet/x64/data'
ydp.genYoloTrnDt(mddf=trainmb[trainmb.batchno == 1], andf=trainad, tgtfld=os.path.join(dataFolder, 'train'))


Processing file (1): VIRAT_S_050300_04_001057_001122.mp4
No. of images written for this video is 1947
Processing file (2): VIRAT_S_050202_07_001126_001301.mp4
No. of images written for this video is 5236
Processing file (3): VIRAT_S_010003_10_000901_000934.mp4
No. of images written for this video is 638
Processing file (4): VIRAT_S_000201_06_001354_001397.mp4
No. of images written for this video is 1284
Processing file (5): VIRAT_S_000201_00_000018_000380.mp4
No. of images written for this video is 10834
Processing file (6): VIRAT_S_050000_16_001947_001989.mp4
No. of images written for this video is 1255
Processing file (7): VIRAT_S_040004_00_000011_000093.mp4
No. of images written for this video is 2449
Processing file (8): VIRAT_S_010106_04_000800_000918.mp4
No. of images written for this video is 2755
Processing file (9): VIRAT_S_010003_06_000526_000560.mp4
No. of images written for this video is 674
Processing file (10): VIRAT_S_040005_06_000886_001016.mp4
No. of images written for

In [14]:
# Testing videos
testmb[testmb.batchno == 1]

,video_name,video_path,object_path,event_path,batchno
0,VIRAT_S_050202_08_001410_001494.mp4,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,1
1,VIRAT_S_000200_06_001693_001824.mp4,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,1
2,VIRAT_S_010002_02_000174_000204.mp4,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,1
3,VIRAT_S_010002_05_000397_000420.mp4,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,None,1
4,VIRAT_S_000205_01_000197_000342.mp4,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,1
5,VIRAT_S_010003_00_000000_000108.mp4,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,1
6,VIRAT_S_010005_08_000647_000693.mp4,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,1
7,VIRAT_S_010204_11_001524_001607.mp4,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,1
8,VIRAT_S_040103_01_000132_000195.mp4,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,1
9,VIRAT_S_010001_02_000195_000498.mp4,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,1


In [15]:
# No. of testing images
print('Total number of testing images that contains bounding box: {}'.format(len(testad[testad.video_name.isin(testmb[testmb.batchno == 1]['video_name'])].groupby(['video_name', 'frame_num']).count())))

Total number of testing images that contains bounding box: 19039


###       Observations:-
    * Total number of images with a person in it is 19,039 
    * Actual number of training images is same as total frame count of all the selected videos and that is 25,754  
    * This also means that we have 25,754 - 19,039 = 6,715 images as negative sales, i.e w/o bounding box and blank annotation file. This is in-line with the recommendation to improve object detection 

In [19]:
# Generate YOLO Test data
dataFolder = 'C:/DS_ML/Video_Analytics_YOLOv4/YOLOv4/darknet/build/darknet/x64/data'
ydp.genYoloTrnDt(mddf=testmb[testmb.batchno == 1], andf=testad, tgtfld=os.path.join(dataFolder, 'test'))


Processing file (1): VIRAT_S_050202_08_001410_001494.mp4
No. of images written for this video is 2512
Processing file (2): VIRAT_S_000200_06_001693_001824.mp4
No. of images written for this video is 3907
Processing file (3): VIRAT_S_010002_02_000174_000204.mp4
No. of images written for this video is 640
Processing file (4): VIRAT_S_010002_05_000397_000420.mp4
No. of images written for this video is 318
Processing file (5): VIRAT_S_000205_01_000197_000342.mp4
No. of images written for this video is 4324
Processing file (6): VIRAT_S_010003_00_000000_000108.mp4
No. of images written for this video is 2589
Processing file (7): VIRAT_S_010005_08_000647_000693.mp4
No. of images written for this video is 862
Processing file (8): VIRAT_S_010204_11_001524_001607.mp4
No. of images written for this video is 1532
Processing file (9): VIRAT_S_040103_01_000132_000195.mp4
No. of images written for this video is 1882
Processing file (10): VIRAT_S_010001_02_000195_000498.mp4
No. of images written for t

### Observations:-
    * Total number of training images    :    49,871
    * Total size of training data        :    19.2 GB
    * Total number of testing images     :    25,754
    * Total size of testing data         :    8.5 GB

In [ ]:
# Training
# !darknet.exe detector train build\\darknet\\x64\\data\\obj.data cfg\\yolov4-obj.cfg C:\\DS_ML\\Video_Analytics_YOLOv4\\YOLOv4\\darknet\\build\\darknet\\x64\\yolov4.conv.137 -map -dont_show

rootDir     =   'C:\\DS_ML\\Video_Analytics_YOLOv4'
darknetDir  =   os.path.join(rootDir, 'YOLOv4', 'darknet')
objData     =   'build\\darknet\\x64\\data\\obj.data'
# cfgFile     =   os.path.join(darknetDir,'cfg\\yolov4-obj.cfg')
cfgFile     =   'cfg\\yolov4-obj.cfg'
weightPath  =   os.path.join(darknetDir, 'build', 'darknet', 'x64', 'yolov4.conv.137')

os.chdir(darknetDir)
command = ['darknet.exe', 'detector', 'train', objData, cfgFile, weightPath, '-map']
train = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, shell=True)
# subprocess.run(command, shell=True)
os.chdir(rootDir)
trnLog = [line.decode().strip() for line in train.stdout.splitlines()]
logpath = os.path.join(rootDir, 'Database_n_Files', 'trainlog.txt')
file = open(logpath, mode='w+')
file.writelines('{}\n'.format(line) for line in trnLog)
file.close()
print('Training log is written in the file {}'.format(logpath))


,video_name,video_path,object_path,event_path,batchno
5,VIRAT_S_050000_16_001947_001989.mp4,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,1
13,VIRAT_S_010201_04_000374_000469.mp4,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,1
20,VIRAT_S_010201_07_000601_000697.mp4,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,C:\DS_ML\Video_Analytics_YOLOv4\VIRAT_Dataset\...,1


In [83]:
# Get Aspect ratio of all the selected videos
vid = ['VIRAT_S_010201_07_000601_000697.mp4','VIRAT_S_010201_04_000374_000469.mp4','VIRAT_S_050000_16_001947_001989.mp4', 'VIRAT_S_010204_11_001524_001607.mp4']
df = pd.concat([trainmb[trainmb.video_name.isin(vid)], testmb[testmb.video_name.isin(vid)]], ignore_index = True)
for _, row in df.iterrows():
    vidcap = cv2.VideoCapture(row['video_path'])
    width = vidcap.get(cv2.CAP_PROP_FRAME_WIDTH)
    height = vidcap.get(cv2.CAP_PROP_FRAME_HEIGHT)
    print('Video: {}'.format(row['video_name']))
    print('Width: {}    Height: {}'.format(width, height))


Video: VIRAT_S_050000_16_001947_001989.mp4
Width: 1920.0    Height: 1080.0
Video: VIRAT_S_010201_04_000374_000469.mp4
Width: 1280.0    Height: 720.0
Video: VIRAT_S_010201_07_000601_000697.mp4
Width: 1280.0    Height: 720.0
Video: VIRAT_S_010204_11_001524_001607.mp4
Width: 1280.0    Height: 720.0


**Its evident that different videos have different width/height. We need to factor this during the process of generation of train/test data from video**

In [92]:
# Pre transformation and normalization on labeling file for a single frame
verdf = trainad[(trainad.video_name == 'VIRAT_S_010201_04_000374_000469.mp4') & (trainad.frame_num == 2241)].copy()
verdf

,obj_id,frame_num,bb_lt_x,bb_lt_y,bb_width,bb_height,obj_type,video_name
98996,18,2241,362,259,25,61,1,VIRAT_S_010201_04_000374_000469.mp4
100788,21,2241,240,225,17,38,1,VIRAT_S_010201_04_000374_000469.mp4
100893,22,2241,263,229,14,23,1,VIRAT_S_010201_04_000374_000469.mp4
101813,23,2241,632,375,54,113,1,VIRAT_S_010201_04_000374_000469.mp4
102056,24,2241,654,321,46,100,1,VIRAT_S_010201_04_000374_000469.mp4
102316,25,2241,684,321,43,93,1,VIRAT_S_010201_04_000374_000469.mp4
103832,27,2241,682,371,52,117,1,VIRAT_S_010201_04_000374_000469.mp4
104596,29,2241,380,252,27,60,1,VIRAT_S_010201_04_000374_000469.mp4


In [104]:
# center
# VIRAT_S_010201_04_000374_000469_2241.txt
center_x = verdf['bb_lt_x'] + verdf['bb_width'] / 2
center_y = verdf['bb_lt_y'] + verdf['bb_height'] / 2
# Normalization 
w= 1280.0
h= 720.0
center_x = center_x / w
center_y = center_y / h
width = verdf['bb_width'] / w
height = verdf['bb_height'] / h
print(pd.DataFrame(list(zip(center_x, center_y, width, height))))

          0         1         2         3
0  0.292578  0.402083  0.019531  0.084722
1  0.194141  0.338889  0.013281  0.052778
2  0.210938  0.334028  0.010937  0.031944
3  0.514844  0.599306  0.042188  0.156944
4  0.528906  0.515278  0.035937  0.138889
5  0.551172  0.510417  0.033594  0.129167
6  0.553125  0.596528  0.040625  0.162500
7  0.307422  0.391667  0.021094  0.083333


In [66]:
# Generate YOLO Training data
dataFolder = 'C:\\DS_ML\\Video_Analytics_YOLOv4\\YOLOv4\\darknet\\build\\darknet\\x64\\data'
trndvid = ['VIRAT_S_010201_07_000601_000697.mp4','VIRAT_S_010201_04_000374_000469.mp4','VIRAT_S_050000_16_001947_001989.mp4']
ydp.genYoloTrnDt(mddf=trainmb[trainmb.video_name.isin(trndvid)], andf=trainad, tgtfld=os.path.join(dataFolder, 'train'))


Processing file (6): VIRAT_S_050000_16_001947_001989.mp4
No. of images written for this video is 1255
Processing file (14): VIRAT_S_010201_04_000374_000469.mp4
No. of images written for this video is 2242
Processing file (21): VIRAT_S_010201_07_000601_000697.mp4
No. of images written for this video is 2208
Image Path is written in the file C:\DS_ML\Video_Analytics_YOLOv4\YOLOv4\darknet\build\darknet\x64\data\train/image_path.txt


In [67]:
# Generate YOLO Test data
dataFolder = 'C:\\DS_ML\\Video_Analytics_YOLOv4\\YOLOv4\\darknet\\build\\darknet\\x64\\data'
ydp.genYoloTrnDt(mddf=testmb[testmb.video_name == 'VIRAT_S_010204_11_001524_001607.mp4'], andf=testad, tgtfld=os.path.join(dataFolder, 'test'))


Processing file (8): VIRAT_S_010204_11_001524_001607.mp4
No. of images written for this video is 1532
Image Path is written in the file C:\DS_ML\Video_Analytics_YOLOv4\YOLOv4\darknet\build\darknet\x64\data\test/image_path.txt


In [65]:
# Test generated Image/frame with that of annotation file

def testgendat(mddf, andf):
    for index, row in mddf.iterrows():
        vidcap = cv2.VideoCapture(row['video_path'])
        cv2.namedWindow('Video', cv2.WINDOW_NORMAL)
        frame_count = 0
        font = cv2.FONT_ITALIC
        while vidcap.isOpened():
            width = vidcap.get(cv2.CAP_PROP_FRAME_WIDTH)
            height = vidcap.get(cv2.CAP_PROP_FRAME_HEIGHT)
            fps = vidcap.get(cv2.CAP_PROP_FPS)
            frm_cnt = vidcap.get(cv2.CAP_PROP_FRAME_COUNT)
            res, frame = vidcap.read()
            if res == True:
                framedf = andf[(andf['video_name'] == row['video_name']) & (andf['frame_num'] == frame_count)].copy()
                framedf['bb_st_co'] = tuple(zip(framedf['bb_lt_x'], framedf['bb_lt_y']))
                framedf['bb_ed_co'] = tuple(zip(framedf['bb_lt_x'] + framedf['bb_width'], framedf['bb_lt_y'] + framedf['bb_height']))
                for _, row in framedf.iterrows():
                    frame = cv2.rectangle(frame, row['bb_st_co'], row['bb_ed_co'], (255, 0, 0), 3)
                frame = cv2.putText(frame, ('w = {}, h = {}'.format(width, height)), (10,50), font, 2, (0, 255, 0),  2)
                frame = cv2.putText(frame, ('fps = {}, frm_cnt = {}, actfrmcnt = {}'.format(fps, frm_cnt, frame_count)), (10,100), font, 1, (0, 255, 0),  2)
                cv2.imshow('Video', frame)
                frame_count += 1
            
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
            else:
                break
        vidcap.release()
        cv2.destroyAllWindows()
        
testgendat(mddf=testmb[testmb.video_name == 'VIRAT_S_010204_11_001524_001607.mp4'], andf=testad)         

In [68]:
import yoloTraining as yt

In [69]:
yt.yolo4Train()

Training log is written in the file C:\DS_ML\Video_Analytics_YOLOv4\Database_n_Files\trainlog.txt


In [71]:
os.chdir('C:\DS_ML\Video_Analytics_YOLOv4\YOLOv4\darknet')

In [72]:
cocoData = 'C:\\DS_ML\\Video_Analytics_YOLOv4\\YOLOv4\\darknet\\build\\darknet\\x64\\data\\obj.data'
cfgFile = 'C:\\DS_ML\\Video_Analytics_YOLOv4\\YOLOv4\\darknet\\cfg\\yolov4-obj.cfg'
weightPath = 'C:\\DS_ML\\Video_Analytics_YOLOv4\\YOLOv4\\darknet\\build\\darknet\\x64\\backup\\yolov4-obj_best.weights'
vidPath = 'C:\\DS_ML\\Video_Analytics_YOLOv4\\VIRAT_Dataset\\videos\\VIRAT_S_010204_11_001524_001607.mp4'
yoloCmd = ['darknet.exe', 'detector', 'demo', cocoData, cfgFile, weightPath, vidPath, '-ext_output']
print(subprocess.run(yoloCmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, shell=True).stdout.decode())

 CUDA-version: 10020 (11000), cuDNN: 8.0.2, CUDNN_HALF=1, GPU count: 1  
 CUDNN_HALF=1 
 OpenCV version: 4.3.0
 0 : compute_capability = 750, cudnn_half = 1, GPU: GeForce RTX 2080 SUPER 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   3 route  1 		                           ->  208 x 208 x  64 
   4 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   5 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   6 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   7 Shortcut Layer: 4,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   8 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   9 route  8 2 	                           ->